# How to use LangChain and Azure OpenAI with Python


Langchain is an open source framework for developing applications using large language models (LLM). <br>

This guide will demonstrate how to setup and use Azure OpenAI models' API with LangChain.
    

## Set Up
The following libraries must be installed to use LangChain with Azure OpenAI.<br>

In [20]:
#%pip install --upgrade openai
#%pip install langchain

## API Configuation and Deployed Model Setup

After installing the necessary libraies, the API must be configured. The code below shows how to configure the API directly in your Python environment. 


In [21]:
import openai
import json
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain import LLMChain


# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base=config_details['OPENAI_API_BASE']
    
# API version e.g. "2023-07-01-preview"
openai_api_version=config_details['OPENAI_API_VERSION']

# The name of your Azure OpenAI deployment chat model. e.g. "gpt-3-turbo-0613"
deployment_name=config_details['DEPLOYMENT_NAME']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# This is set to `azure`
openai_api_type="azure"

## Deployed Model Setup

In [22]:
# Create an instance of chat llm
llm = AzureChatOpenAI(
    openai_api_base=openai_api_base,
    openai_api_version=openai_api_version,
    deployment_name=deployment_name,
    openai_api_key=openai_api_key,
    openai_api_type=openai_api_type,
)

llm([HumanMessage(content="Write me a poem")])

AIMessage(content="In a world of dreams, where words take flight,\nI'll weave a poem, to bring you delight.\nWith ink and rhythm, I'll paint a scene,\nWhere emotions dance in colors unseen.\n\nIn twilight's embrace, the stars ignite,\nAs moonbeams whisper, a poet's plight.\nEach syllable, a gentle caress,\nUnveiling secrets, in poetic finesse.\n\nThrough nature's canvas, I'll take you there,\nWhere sunsets melt, in hues so rare.\nA symphony of birds, their melodies blend,\nAs whispers of wind, through tall trees ascend.\n\nIn the depths of silence, where echoes reside,\nI'll find the stories that time can't hide.\nUnearth forgotten tales, from ancient lore,\nAnd breathe life into them, like never before.\n\nWith passion's fire, my verses will ignite,\nEmbracing love's joy, and sorrow's might.\nI'll capture the essence, of a tender touch,\nAnd the ache of longing, that hurts so much.\n\nThrough valleys of hope, and mountains of despair,\nI'll guide you gently, with tender care.\nWith ea

## PromptTemplates

Langchain provides a built in PromptsTemplate module to simplify the construction of prompts to get more specific answers.

In [23]:
from langchain import PromptTemplate

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


# First Example
template = """
You are a skin care consulant that recommends products based on customer
needs and preferences.

What is a good {product_type} to help with {customer_request}?
"""

prompt = PromptTemplate(
input_variables=["product_type", "customer_request"],
template=template,
)

print("Example #1:")
print(llm([HumanMessage(content=prompt.format(
        product_type="face wash",
        customer_request = "acne prone skin"
    ))]
))
print("\n")

# Second Example
system_message = "You are an AI assistant travel assistant that provides vacation recommendations."

system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt)
result = chain.run(f"Where should I go on vaction in Decemember for warm weather and beaches?")
print("Example #2:")
print(result)

Example #1:
content="A good face wash for acne-prone skin is one that contains ingredients like salicylic acid or benzoyl peroxide. These ingredients are effective in combating acne by unclogging pores, reducing inflammation, and killing acne-causing bacteria. Here are a few recommendations:\n\n1. Neutrogena Oil-Free Acne Wash: This face wash contains salicylic acid, which helps to clear breakouts and prevent new ones. It is gentle on the skin and doesn't leave it feeling dry or tight.\n\n2. La Roche-Posay Effaclar Medicated Gel Cleanser: This cleanser is formulated with 2% salicylic acid and helps to remove excess oil and impurities from the skin. It is suitable for sensitive skin and helps to reduce acne breakouts.\n\n3. CeraVe Acne Foaming Cream Cleanser: This face wash is formulated with benzoyl peroxide to combat acne and promote a clear complexion. It also contains ceramides and hyaluronic acid to moisturize and protect the skin barrier.\n\nRemember, it's important to choose a fa

## Chains
There are many applications of chains that allow you to combine numerous LLM calls and actions.  <br>

### Simple Sequential Chains  <br>
Allow you to feed the output of one LLM Chain as input for another.

In [24]:
from langchain.chains import SimpleSequentialChain

In [25]:
template = """Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.
% CHILD_DESCRIPTION
{child_description}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["child_description"], template=template)

# Holds my 'location' chain
description_chain = LLMChain(llm=llm, prompt=prompt_template)

In [26]:
template = """Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.
% DIY_PROJECT
{diy_project}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["diy_project"], template=template)

# Holds my 'meal' chain
diy_chain = LLMChain(llm=llm, prompt=prompt_template)

In [27]:
overall_chain = SimpleSequentialChain(chains=[description_chain, diy_chain], verbose=True)

In [28]:
review = overall_chain.run("5-year-old girl")



> Entering new SimpleSequentialChain chain...
DIY Princess Crown Craft Kit: 

Materials needed:
- Cardboard or foam sheet
- Scissors
- Glue
- Decorative items (such as glitter, stickers, feathers, beads, sequins)
- Elastic string or ribbon

Instructions:
1. Cut a strip of cardboard or foam sheet that is long enough to fit around the child's head.
2. Measure the strip to the child's head size and cut to adjust if needed.
3. Let the child decorate the strip with glue and decorative items of their choice. They can use glitter, stickers, feathers, beads, sequins, or anything else they find appealing.
4. Once the decorations are complete, let the glue dry completely.
5. Attach elastic string or ribbon to both ends of the strip to create a crown that can be worn on the head.
6. Help the child tie the elastic string or ribbon securely, making sure it is not too tight or too loose.
7. The princess crown is now ready to be worn and enjoyed! The child can wear it during imaginative play, dress